In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout
from keras.layers import Dense
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Model
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/binancecoin-data/BNB-USD.csv')
data.head()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data = data[['Date', 'Close']]

In [ ]:
plt.figure(figsize=(15, 5))
sns.lineplot(x = data['Date'], y = data['Close'])

In [ ]:
data.dropna(inplace=True)

In [ ]:
close_price = data['Close'].values.astype('float32')

In [ ]:
close_price = close_price.reshape(1481, 1)

In [ ]:
scaler = StandardScaler()
close_price = scaler.fit_transform(close_price)

In [ ]:
train_size = int(len(close_price) * 0.80)
test_size = len(close_price) - train_size
train, test = close_price[0:train_size,:], close_price[train_size:len(close_price),:]
print(len(train), len(test))

In [ ]:
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
X_train =trainX.reshape(trainX.shape[0],trainX.shape[1] , 1)
X_test = testX.reshape(testX.shape[0],testX.shape[1] , 1)

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(10,1)))
model.add(LSTM(50,return_sequences=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, trainY, epochs = 10, batch_size = 32, validation_split=0.2)

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
trainy = scaler.inverse_transform(trainY).reshape(-1, 1)

In [ ]:
train_predict = train_predict.reshape(-1, 1)

In [ ]:
trainy.mean()

In [ ]:
train_predict.mean()